In [1]:
!git clone https://github.com/PanQiWei/AutoGPTQ.git

fatal: destination path 'AutoGPTQ' already exists and is not an empty directory.


In [2]:
%cd AutoGPTQ
!pip install numpy gekko pandas

/content/AutoGPTQ


In [3]:
!pip install -vvv --no-build-isolation -e .

Using pip 24.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Non-user install because site-packages writeable
Created temporary directory: /tmp/pip-build-tracker-wlkt3p8q
Initialized build tracking at /tmp/pip-build-tracker-wlkt3p8q
Created build tracker: /tmp/pip-build-tracker-wlkt3p8q
Entered build tracker: /tmp/pip-build-tracker-wlkt3p8q
Created temporary directory: /tmp/pip-install-wsuoqhcv
Created temporary directory: /tmp/pip-ephem-wheel-cache-fm85927d
Obtaining file:///content/AutoGPTQ
  Added file:///content/AutoGPTQ to build tracker '/tmp/pip-build-tracker-wlkt3p8q'
  Running setup.py (path:/content/AutoGPTQ/setup.py) egg_info for package from file:///content/AutoGPTQ
  Created temporary directory: /tmp/pip-pip-egg-info-u2am1z56
  Running command python setup.py egg_info
  conda_cuda_include_dir /usr/lib/python3/dist-packages/nvidia/cuda_runtime/include
  running egg_info
  creating /tmp/pip-pip-egg-info-u2am1z56/auto_gptq.egg-info
  writing /tmp/pip-pip-egg

In [5]:
!python ./examples/quantization/basic_usage_wikitext2.py

README.md: 100% 10.5k/10.5k [00:00<00:00, 54.3MB/s]
test-00000-of-00001.parquet: 100% 733k/733k [00:00<00:00, 99.0MB/s]
train-00000-of-00001.parquet: 100% 6.36M/6.36M [00:00<00:00, 209MB/s]
validation-00000-of-00001.parquet: 100% 657k/657k [00:00<00:00, 455MB/s]
Generating test split: 100% 4358/4358 [00:00<00:00, 112903.74 examples/s]
Generating train split: 100% 36718/36718 [00:00<00:00, 718947.46 examples/s]
Generating validation split: 100% 3760/3760 [00:00<00:00, 599984.14 examples/s]
tokenizer_config.json: 100% 685/685 [00:00<00:00, 5.05MB/s]
config.json: 100% 651/651 [00:00<00:00, 5.17MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 4.08MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 2.15MB/s]
special_tokens_map.json: 100% 441/441 [00:00<00:00, 4.32MB/s]
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4

In [ ]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the LLaMA-2 model name, for example, "meta-llama/Llama-2-7b-hf"
# You can choose different model sizes: "meta-llama/Llama-2-7b-hf", "meta-llama/Llama-2-13b-hf", "meta-llama/Llama-2-70b-hf"
list_model_names = ["meta-llama/Llama-2-7b-hf", "meta-llama/Llama-2-13b-hf", "meta-llama/Llama-2-70b-hf"]
model_name = list_model_names[1]  # Select LLaMA-2 13B model
model_type = os.path.basename(model_name)

# Load the LLaMA-2 model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Save the model and tokenizer locally
model.save_pretrained("./llama2_model_13b")
tokenizer.save_pretrained("./llama2_model_13b")

print(f"Model and tokenizer for {model_name} have been downloaded and saved locally.")

In [12]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

# Specify the model name, for example, "facebook/opt-125m" (you can choose different model sizes)
list_model_names = ["facebook/opt-1.3b", "facebook/opt-2.7b", "facebook/opt-6.7b", "facebook/opt-13b", "facebook/opt-30b"]
model_name = list_model_names[1]
model_type = os.path.basename(model_name)
# variants: facebook/opt-350m, facebook/opt-700m, facebook/opt-1.3b, facebook/opt-2.7b, facebook/opt-6.7b, facebook/opt-13b, facebook/opt-30b
# variants:

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Save the model and tokenizer locally
model.save_pretrained("./opt_model_2.7b")
tokenizer.save_pretrained("./opt_model_2.7b")

print(f"Model and tokenizer for {model_name} have been downloaded and saved locally.")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Model and tokenizer for facebook/opt-2.7b have been downloaded and saved locally.


In [7]:
import numpy as np
import torch
import torch.nn as nn

from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

In [8]:
# os.makedirs(quantized_model_dir, exist_ok=True)
def get_wikitext2(nsamples, seed, seqlen, model):
    from datasets import load_dataset

    traindata = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
    testdata = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")

    from transformers import AutoTokenizer

    try:
        tokenizer = AutoTokenizer.from_pretrained(model, use_fast=False)
    except Exception:
        tokenizer = AutoTokenizer.from_pretrained(model, use_fast=True)
    trainenc = tokenizer("\n\n".join(traindata["text"]), return_tensors="pt")
    testenc = tokenizer("\n\n".join(testdata["text"]), return_tensors="pt")

    import random

    random.seed(seed)
    np.random.seed(0)
    torch.random.manual_seed(0)

    traindataset = []
    for _ in range(nsamples):
        i = random.randint(0, trainenc.input_ids.shape[1] - seqlen - 1)
        j = i + seqlen
        inp = trainenc.input_ids[:, i:j]
        attention_mask = torch.ones_like(inp)
        traindataset.append({"input_ids": inp, "attention_mask": attention_mask})
    return traindataset, testenc

In [9]:
@torch.no_grad()
def opt_eval(model, testenc, dev, seqlen=2048):
    print("Evaluating ...")

    testenc = testenc.input_ids
    nsamples = testenc.numel() // seqlen

    use_cache = model.config.use_cache
    model.config.use_cache = False
    layers = model.model.decoder.layers

    model.model.decoder.embed_tokens = model.model.decoder.embed_tokens.to(dev)
    model.model.decoder.embed_positions = model.model.decoder.embed_positions.to(dev)
    if hasattr(model.model.decoder, "project_out") and model.model.decoder.project_out:
        model.model.decoder.project_out = model.model.decoder.project_out.to(dev)
    if hasattr(model.model.decoder, "project_in") and model.model.decoder.project_in:
        model.model.decoder.project_in = model.model.decoder.project_in.to(dev)
    layers[0] = layers[0].to(dev)

    dtype = next(iter(model.parameters())).dtype
    inps = torch.zeros((nsamples, seqlen, model.config.hidden_size), dtype=dtype, device=dev)
    cache = {"i": 0, "attention_mask": None}

    class Catcher(nn.Module):
        def __init__(self, module):
            super().__init__()
            self.module = module

        def forward(self, inp, **kwargs):
            inps[cache["i"]] = inp
            cache["i"] += 1
            cache["attention_mask"] = kwargs["attention_mask"]
            raise ValueError

    layers[0] = Catcher(layers[0])
    for i in range(nsamples):
        batch = testenc[:, (i * seqlen) : ((i + 1) * seqlen)].to(dev)
        try:
            model(batch)
        except ValueError:
            pass
    layers[0] = layers[0].module

    layers[0] = layers[0].cpu()
    model.model.decoder.embed_tokens = model.model.decoder.embed_tokens.cpu()
    model.model.decoder.embed_positions = model.model.decoder.embed_positions.cpu()
    if hasattr(model.model.decoder, "project_out") and model.model.decoder.project_out:
        model.model.decoder.project_out = model.model.decoder.project_out.cpu()
    if hasattr(model.model.decoder, "project_in") and model.model.decoder.project_in:
        model.model.decoder.project_in = model.model.decoder.project_in.cpu()
    torch.cuda.empty_cache()

    outs = torch.zeros_like(inps)
    attention_mask = cache["attention_mask"]

    for i in range(len(layers)):
        print(i)
        layer = layers[i].to(dev)

        for j in range(nsamples):
            outs[j] = layer(inps[j].unsqueeze(0), attention_mask=attention_mask)[0]
        layers[i] = layer.cpu()
        del layer
        torch.cuda.empty_cache()
        inps, outs = outs, inps

    if model.model.decoder.final_layer_norm is not None:
        model.model.decoder.final_layer_norm = model.model.decoder.final_layer_norm.to(dev)
    if model.model.decoder.project_out is not None:
        model.model.decoder.project_out = model.model.decoder.project_out.to(dev)
    model.lm_head = model.lm_head.to(dev)

    testenc = testenc.to(dev)
    nlls = []
    for i in range(nsamples):
        hidden_states = inps[i].unsqueeze(0)
        if model.model.decoder.final_layer_norm is not None:
            hidden_states = model.model.decoder.final_layer_norm(hidden_states)
        if model.model.decoder.project_out is not None:
            hidden_states = model.model.decoder.project_out(hidden_states)
        lm_logits = model.lm_head(hidden_states)
        shift_logits = lm_logits[:, :-1, :].contiguous()
        shift_labels = testenc[:, (i * seqlen) : ((i + 1) * seqlen)][:, 1:]
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        neg_log_likelihood = loss.float() * seqlen
        nlls.append(neg_log_likelihood)
    ppl = torch.exp(torch.stack(nlls).sum() / (nsamples * seqlen))
    print(ppl.item())

    model.config.use_cache = use_cache

In [ ]:
pretrained_model_dir = "./opt_model_2.7b"
quantized_model_dir = "opt-2.7b-4bit-128g"

traindataset, testenc = get_wikitext2(128, 0, 2048, pretrained_model_dir)

quantize_config = BaseQuantizeConfig(
    bits=4,  # quantize model to 4-bit
    group_size=128,  # it is recommended to set the value to 128
    desc_act=False,  # desc_act and group size only works on triton
)

# load un-quantized model, the model will always be force loaded into cpu
model = AutoGPTQForCausalLM.from_pretrained(pretrained_model_dir, quantize_config)

# quantize model, the examples should be list of dict whose keys can only be "input_ids" and "attention_mask"
# with value under torch.LongTensor type.
model.quantize(traindataset, use_triton=False)

# save quantized model
model.save_quantized(quantized_model_dir)

# save quantized model using safetensors
model.save_quantized(quantized_model_dir, use_safetensors=True)

# load quantized model, currently only support cpu or single gpu
model = AutoGPTQForCausalLM.from_quantized(quantized_model_dir, device="cuda:0", use_triton=False)

opt_eval(model.model, testenc, "cuda:0")
